In [20]:
import pandas as pd
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from time import strftime, strptime, mktime

In [21]:
df = pd.read_csv('./train.csv')
df.shape

(69713, 22)

In [22]:
test = pd.read_csv('./test.csv')
test.shape

(30037, 21)

In [23]:
y = df.pop('Approved')

In [24]:
train_id = df.pop('ID')
test_id = test.pop('ID')

In [25]:
def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

In [26]:
%%time
df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

CPU times: user 1.91 s, sys: 161 ms, total: 2.07 s
Wall time: 2.07 s


In [27]:
%%time
df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

CPU times: user 56 ms, sys: 2 ms, total: 58 ms
Wall time: 55.7 ms


In [28]:
%%time
df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

CPU times: user 7.8 s, sys: 2.93 s, total: 10.7 s
Wall time: 10.7 s


In [29]:
df.shape, test.shape

((69713, 37416), (30037, 37416))

In [30]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler

In [31]:
s = MinMaxScaler()
s

MinMaxScaler(copy=True, feature_range=(0, 1))

In [32]:
%%time
df = s.fit_transform(df)
test = s.transform(test)

CPU times: user 34.5 s, sys: 30.4 s, total: 1min 4s
Wall time: 1min 4s


In [33]:
%%time
m = lambda *x: [i.shape for i in x]
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
print m(x_train, y_train, x_test, y_test)

[(55770, 37416), (55770,), (13943, 37416), (13943,)]
CPU times: user 26.7 s, sys: 11.6 s, total: 38.3 s
Wall time: 38.3 s


In [34]:
from sklearn.ensemble import RandomForestClassifier


In [35]:
model = RandomForestClassifier(n_estimators=600, min_samples_split = 5, n_jobs=24)
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=24,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [36]:
%%time
model.fit(x_train ,y_train)

CPU times: user 2h 31min 50s, sys: 2.98 s, total: 2h 31min 53s
Wall time: 6min 35s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=24,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [37]:
pred = model.predict(x_test)
predb = model.predict_proba(x_test)

prob = predb[:,1]
roc_auc_score(y_test, prob)

0.8303719363808864


600 6 0.8224773007456205

600 5 std 0.8404221862156795

600 5 mn 0.835911554973441

In [32]:
f_model = RandomForestClassifier(n_estimators=600, n_jobs=24, min_samples_split=5)
f_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=24,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
%%time
f_model.fit(df ,y)

CPU times: user 1h 44min 20s, sys: 3.26 s, total: 1h 44min 23s
Wall time: 4min 32s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=24,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [34]:
model = f_model

___

In [35]:
%%time
prob_test = model.predict_proba(test)

CPU times: user 24.7 s, sys: 841 ms, total: 25.5 s
Wall time: 3.22 s


In [36]:
prob_s = prob_test[:,1]
prob_s

array([0.00386508, 0.04251419, 0.00330556, ..., 0.        , 0.00638889,
       0.03449272])

In [37]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(prob_s, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [38]:
submit.to_csv('fr600.t5_f_std.csv',index=False)

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba